In [ ]:
# @title Set up

import os
import io
import glob
import json
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import glob
import argparse
from imblearn.under_sampling import NearMiss
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.dummy import DummyClassifier
from sklearn.metrics import make_scorer, accuracy_score, roc_auc_score
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, make_scorer, accuracy_score, roc_auc_score
from sklearn.model_selection import cross_validate


from google.colab import drive, files
drive.mount('/content/drive')

!pip install pandas==2.0.3
!pip install nilearn
!pip install mne
!pip install -U mne-connectivity

import pandas as pd

In [ ]:
#@title Data

df = pd.read_csv('/content/drive/MyDrive/Favaloro/Tesis/vectores.csv')
path_ABIDE = "/content/drive/MyDrive/Favaloro/Tesis/ABIDE"
coords = pd.read_csv(os.path.join(path_ABIDE, "rois_coordinates.csv"))
roi_coords = coords[['x', 'y', 'z']].values

### Agregando las ROIs

In [ ]:
# TBD no poner los nombres en español, sino que mantener los nombres originales del atlas




roi_labels = np.array([
"Precentral Izq", "Precentral Der", "Frontal Superior 2 Izq",
"Frontal Superior 2 Der", "Frontal Media 2 Izq", "Frontal Media 2 Der",
"Frontal Inferior Opercular Izq", "Frontal Inferior Opercular Der",
"Frontal Inferior Triangular Izq", "Frontal Inferior Triangular Der",
"Frontal Inferior Orbital 2 Izq", "Frontal Inferior Orbital 2 Der",
"Rolándica Opercular Izq", "Rolándica Opercular Der", "Área Motora Suplementaria Izq",
"Área Motora Suplementaria Der", "Olfactoria Izq", "Olfactoria Der",
"Frontal Superior Medial Izq", "Frontal Superior Medial Der",
"Frontal Medial Orbital Izq", "Frontal Medial Orbital Der", "Recto Izq",
"Recto Der", "Corteza Orbitofrontal medial Izq", "Corteza Orbitofrontal medial Der",
"Corteza Orbitofrontal Cantonal Izq", "Corteza Orbitofrontal Cantonal Der",
"Corteza Orbitofrontal Posterior Izq", "Corteza Orbitofrontal Posterior Der",
"Corteza Orbitofrontal Lateral Izq", "Corteza Orbitofrontal Lateral Der",
"Ínsula Izq", "Ínsula Der", "Cíngulo Anterior Izq", "Cíngulo Anterior Der",
"Cíngulo Medio Izq", "Cíngulo Medio Der", "Cíngulo Posterior Izq",
"Cíngulo Posterior Der", "Hipocampo Izq", "Hipocampo Der",
"Parahipocampal Izq", "Parahipocampal Der", "Amígdala Izq",
"Amígdala Der", "Calcarina Izq", "Calcarina Der", "Cuneus Izq",
"Cuneus Der", "Lingual Izq", "Lingual Der", "Occipital Superior Izq",
"Occipital Superior Der", "Occipital Medio Izq", "Occipital Medio Der",
"Occipital Inferior Izq", "Occipital Inferior Der", "Fusiforme Izq",
"Fusiforme Der", "Postcentral Izq", "Postcentral Der",
"Parietal Superior Izq", "Parietal Superior Der", "Parietal Inferior Izq",
"Parietal Inferior Der", "Supramarginal Izq", "Supramarginal Der",
"Angular Izq", "Angular Der", "Precuneus Izq", "Precuneus Der",
"Lobulo Paracentral Izq", "Lobulo Paracentral Der", "Caudado Izq",
"Caudado Der", "Putamen Izq", "Putamen Der", "Pálido Izq",
"Pálido Der", "Tálamo Izq", "Tálamo Der", "Heschl Izq",
"Heschl Der", "Temporal Superior Izq", "Temporal Superior Der",
"Polo Temporal Superior Izq", "Polo Temporal Superior Der",
"Temporal Medio Izq", "Temporal Medio Der", "Polo Temporal Medio Izq",
"Polo Temporal Medio Der", "Temporal Inferior Izq", "Temporal Inferior Der"
])


In [ ]:
# Construir un diccionario de características (i, j) -> 'ROI_label_i - ROI_label_j'

feature_labels = {}
k = 0
for i in range(len(roi_labels)):
    for j in range(i+1, len(roi_labels)):
        # Asegúrate de que el formato del nombre coincide exactamente con cómo se forman en df
        feature_labels[f'{roi_labels[i]} - {roi_labels[j]}'] = k
        k += 1

# Verifica que las claves de feature_labels se formen correctamente
print(list(feature_labels.keys())[:10])  # Imprime las primeras 10 claves para verificar


### Train Test split

In [ ]:
# Invertir el diccionario para mapear de índices a nombres de características
inverse_feature_labels = {v: k for k, v in feature_labels.items()}

# Ahora inverse_feature_labels es un diccionario donde las claves son índices numéricos y los valores son los nombres de las características


In [ ]:
# Crear una lista de los nuevos nombres de las columnas usando el diccionario invertido
new_column_names = [inverse_feature_labels[i] for i in range(len(df.columns) - 1)]  # Excluye la última columna
new_column_names.append('condicion')  # Añade 'condicion' al final para mantenerla

# Asignar los nuevos nombres a las columnas del DataFrame
df.columns = new_column_names


In [ ]:
print(df.columns)


In [ ]:
# Por convención
# Controles = cero: 0
# Autistas = uno: 1

df["condicion"] = df["condicion"].replace("CONT", 0)

df["condicion"] = df["condicion"].replace("AUT", 1)

In [ ]:
features = df.drop(['condicion'], axis=1)
condicion = df['condicion'].values
features.shape

In [ ]:
# División de los datos en conjuntos de entrenamiento y prueba

from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(features, condicion, stratify = condicion, test_size=0.25)
# Stratify balancea la cantidad de sujetos de cada condicion en la división de os datos

print(f'X_train: {X_train.shape}')
print(f'X_test: {X_test.shape}')
print(f'y_train: {y_train.shape}')
print(f'y_test: {y_test.shape}')

In [ ]:
# Porcentaje de autistas en el conjunto de entrenamiento
porcentaje_autistas_train = (y_train == 1).sum() / len(y_train) * 100

"""
# Porcentaje de autistas en el conjunto de entrenamiento
porcentaje_ytrain_res = (y_train == 1).sum() / len(y_train_res) * 100
porcentaje_xtrain_res = (y_train == 1).sum() / len(X_train_res) * 100
"""
# Porcentaje de controles en el conjunto de entrenamiento
porcentaje_controles_train = (y_train == 0).sum() / len(y_train) * 100

# Porcentaje de autistas en el conjunto de prueba
porcentaje_autistas_test = (y_test == 1).sum() / len(y_test) * 100

# Porcentaje de controles en el conjunto de prueba
porcentaje_controles_test = (y_test == 0).sum() / len(y_test) * 100



print(f'Porcentaje de autistas en el conjunto de entrenamiento: {porcentaje_autistas_train:.2f}%')
print(f'Porcentaje de controles en el conjunto de entrenamiento: {porcentaje_controles_train:.2f}%')
print(f'Porcentaje de autistas en el conjunto de prueba: {porcentaje_autistas_test:.2f}%')
print(f'Porcentaje de controles en el conjunto de prueba: {porcentaje_controles_test:.2f}%')
#print(porcentaje_ytrain_res)
#print(porcentaje_xtrain_res)


In [ ]:
# Dataframe para los distintos modelos
columns = ['Model Type', 'Hyperparameters', 'Accuracy', 'AUC', 'Precision Class 0', 'Precision Class 1', 'Recall Class 0', 'Recall Class 1', 'F1-Score Class 0', 'F1-Score Class 1', 'Coefficients', 'Probabilities']
results_df = pd.DataFrame(columns=columns)

In [ ]:
# Función para agregar modelos al dataframe
def add_model_results(results_df, model, report, accuracy, auc):
    precision_0 = report['0']['precision']
    precision_1 = report['1']['precision']
    recall_0 = report['0']['recall']
    recall_1 = report['1']['recall']
    f1_0 = report['0']['f1-score']
    f1_1 = report['1']['f1-score']

    # Asumimos que el modelo puede predecir probabilidades y tiene coeficientes
    coefs = [model.coef_.ravel()]
    probabilities = model.predict_proba(X_test)  # Asegúrate de que X_test esté definido en tu contexto

    # Extraer hiperparámetros directamente del modelo
    hyperparams = model.get_params()

    # Construir un diccionario con la información del modelo
    model_info = pd.DataFrame({
        'Model Type': [type(model).__name__],
        'Hyperparameters': [str(hyperparams)],
        'Accuracy': [accuracy],
        'AUC': [auc],
        'Precision Class 0': [precision_0],
        'Precision Class 1': [precision_1],
        'Recall Class 0': [recall_0],
        'Recall Class 1': [recall_1],
        'F1-Score Class 0': [f1_0],
        'F1-Score Class 1': [f1_1],
        'Coefficients': [coefs],
        'Probabilities': [probabilities.tolist()]
    })

    # Utilizar pd.concat para añadir la información del nuevo modelo al DataFrame existente
    results_df = pd.concat([results_df, model_info], ignore_index=True)
    return results_df


# ejemplo para llamar a la funcion
# results_df = add_model_results(results_df, best_model, report, cv_results['test_accuracy'][best_index], cv_results['test_AUC'].mean(), hyperparams, coefs_array, probabilities)

### CV strategy

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

#from sklearn.model_selection import ShuffleSplit

#cv_generator = ShuffleSplit(n_splits=100, test_size=0.3, random_state=42)
cv_generator = StratifiedShuffleSplit(n_splits=100, test_size=0.3, random_state=42)

### Dummy

In [ ]:
from sklearn.dummy import DummyClassifier
dummy_clf = DummyClassifier(strategy="stratified")

dummy_clf.fit(X_train, y_train)
y_pred = dummy_clf.predict(X_test)

baseline_accuracy = dummy_clf.score(X_test, y_test)
print(f"Baseline accuracy: {baseline_accuracy}")

In [ ]:
conf_mat = confusion_matrix(y_test, y_pred)
print(conf_mat)
sns.heatmap(conf_mat, annot = True, fmt = 'd', cmap= 'Blues')
plt.xlabel('Predicciones del modelo')
plt.ylabel('Condición real')
plt.show()

# Logistic Regression with Stratified Shuffle Split

In [ ]:
# este sería el codigo para estandarizar los datos, que se supone crucial para poder interpretar correctamente los valores de coefs
# ver de aplicarlo y comparar rendimiento

# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import cross_validate, StratifiedKFold
# from sklearn.metrics import accuracy_score, make_scorer, roc_auc_score, classification_report
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import StandardScaler
# import pandas as pd
# import numpy as np

# # Definir el modelo con los hiperparámetros elegidos
# LR = LogisticRegression(C=0.1, max_iter=1000, solver='newton-cg')

# # Crear un pipeline que incluya estandarización y el modelo
# pipeline = Pipeline([
#     ('scaler', StandardScaler()),
#     ('logreg', LR)
# ])

# # Definir las métricas para la validación cruzada
# scoring = {
#     'accuracy': make_scorer(accuracy_score),
#     'AUC': 'roc_auc'
# }

# # Crear el generador de pliegues para la validación cruzada
# cv_generator = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# # Realizar la validación cruzada y guardar los estimadores
# cv_results = cross_validate(pipeline, X_train, y_train, scoring=scoring, cv=cv_generator, n_jobs=1, verbose=1, return_train_score=True, return_estimator=True)

# coefs = []

# # Iterar a través de los estimadores y extraer los coeficientes
# for estimator in cv_results['estimator']:
#     coefs.append(estimator.named_steps['logreg'].coef_.ravel())

# coefs_array = np.array(coefs)

# # Crear un DataFrame para los coeficientes y guardarlos en un archivo Excel
# coefs_df = pd.DataFrame(coefs_array)
# coefs_df.to_excel('coeficientes_cv.xlsx', index=False)

# # Imprimir la precisión promedio y las puntuaciones AUC de la validación cruzada
# print(f"Average Accuracy: {cv_results['test_accuracy'].mean()}")
# print(f"Average AUC: {cv_results['test_AUC'].mean()}")

# # Encontrar el índice del mejor modelo
# best_index = np.argmax(cv_results['test_accuracy'])

# # Seleccionar el mejor modelo
# best_model = cv_results['estimator'][best_index]

# # Aplicar predict_proba en el conjunto de prueba
# probabilities = best_model.predict_proba(X_test)

# # Predecir las etiquetas del conjunto de prueba
# y_pred = best_model.predict(X_test)

# # Generar el informe de clasificación
# report = classification_report(y_test, y_pred, output_dict=True)

# # Resultados
# result = {
#     'Model': best_model,
#     'Accuracy': cv_results['test_accuracy'][best_index],
#     'Classification Report': report,
#     'Predictions Probabilities': probabilities
# }

# print("Model:", result['Model'])
# print("Accuracy:", result['Accuracy'])
# print("Classification Report:", result['Classification Report'])


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, make_scorer, roc_auc_score
from sklearn.metrics import classification_report

# Modelo LR con hiperparametros elegidos en base a pruebas anteriores

LR = LogisticRegression(C=0.1, max_iter=1000, solver='newton-cg')

# Métricas para la CV
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'AUC': 'roc_auc'
}

# Se realiza CV y se guardan los estimadores
cv_results = cross_validate(LR, X_train, y_train, scoring=scoring, cv=cv_generator, n_jobs=1, verbose=1, return_train_score=True, return_estimator=True)


coefs = []

# Iterar a través de los estimadores y extraer los coeficientes
for estimator in cv_results['estimator']:
    coefs.append(estimator.coef_.ravel())

coefs_array = np.array(coefs)

# Transpose the array so that each column represents a set of coefficients from one iteration of CV
# coefs_array = coefs_array.T

coefs_df = pd.DataFrame(coefs_array)
coefs_df.to_excel('coeficientes_cv.xlsx', index=False)

# Print the average accuracy and AUC scores from cross-validation
print(f"Average Accuracy: {cv_results['test_accuracy'].mean()}")
print(f"Average AUC: {cv_results['test_AUC'].mean()}")


# Encontrar el índice del mejor modelo
best_index = np.argmax(cv_results['test_accuracy'])

# Seleccionar el mejor modelo
best_model = cv_results['estimator'][best_index]

# Aplicar predict_proba en el conjunto de prueba
probabilities = best_model.predict_proba(X_test)

# Predice las etiquetas del conjunto de prueba
y_pred = best_model.predict(X_test)


# Informe de clasificación
report = classification_report(y_test, y_pred, output_dict=True)

# Resultados
result = {
    'Model': best_model,
    'Accuracy': cv_results['test_accuracy'][best_index],
    'Classification Report': report,
    'Predictions Probabilities': probabilities
}

print("Model:", result['Model'])
print("Accuracy:", result['Accuracy'])
print("Classification Report:", result['Classification Report'])


# print(result)


In [ ]:
# Matriz de confusión con las predicciónes del mejor modelo obtenido
conf_mat = confusion_matrix(y_test, y_pred)
print(conf_mat)
sns.heatmap(conf_mat, annot = True, fmt = 'd', cmap= 'Blues')
plt.xlabel('Predicciones del modelo')
plt.ylabel('Condición real')
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc

y_score = probabilities[:, 1]

# Calcular los valores de tasa de verdaderos positivos (TPR), tasa de falsos positivos (FPR) y los umbrales
fpr, tpr, thresholds = roc_curve(y_test, y_score)

# Calcular el área bajo la curva ROC (AUC)
roc_auc = auc(fpr, tpr)

# Graficar la curva ROC
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
results_df = add_model_results(results_df, best_model, report, cv_results['test_accuracy'][best_index], cv_results['test_AUC'].mean())

In [ ]:
print(results_df.head())

In [ ]:
results_df.to_csv('results_1', index=False)

In [ ]:
# Copiar nombres de columnas desde `df` excluyendo 'condicion' si es aplicable
coefs_df.columns = df.columns[:-1]  # Excluye 'condicion' si `df` incluye esta columna
coefs_df.to_csv('coefs_1.csv', index=False)

# Descargar el archivo
files.download('coefs_1.csv')

In [ ]:
coefs_df.head()

In [ ]:
# https://forecastegy.com/posts/feature-importance-in-logistic-regression/
# Convertir la lista de coeficientes en una matriz
coefs_matrix = np.array(coefs_df)

# Calcular la mediana para cada columna
median_coefs = np.median(coefs_matrix, axis=0)

# Los coeficientes representan el cambio en el logaritmo de odds por un incremento unitario en la característica correspondiente,
# y por lo tanto, su magnitud (ignorando el signo) puede interpretarse como la "importancia" de la característica en la predicción del modelo.
# Cuando las características están estandarizadas, la magnitud de los coeficientes en la regresión logística refleja directamente la importancia de cada característica.
# en base a lo anterior, tendría sentido tambien analizar las tail(10), entendiendolas como las que menos importancia tienen?
#(seria que esas tambien representan hipoactivacion?)

# Crear DataFrame de importancias
feature_importances = pd.DataFrame({
    'Feature': coefs_df.columns,  # TBD corroborar que realmente se mantenga el orden original
    'Importance': np.abs(median_coefs)
})

# Ordenar las características por importancia
sorted_feature_importances = feature_importances.sort_values(by='Importance', ascending=False)
# Extraer las 10 características más importantes con sus importancias
top_10_features = sorted_feature_importances.head(10)


top_10_features.to_csv('top_10_features_1.csv', index=False)
files.download('top_10_features_1.csv')

# Visualizar las importancias ordenadas
print(sorted_feature_importances)
print(top_10_features)

In [ ]:
# Configurar el tamaño y estilo del gráfico
plt.figure(figsize=(10, 8))
sns.barplot(x='Importance', y='Feature', data=top_10_features, palette='viridis')

# Títulos y etiquetas
plt.title('Top 10 Most Important Features')
plt.xlabel('Importance (Median Absolute Coefficient)')
plt.ylabel('Features')

# Mostrar el gráfico
plt.show()

In [ ]:
plt.figure(figsize=(15, 20))

for i, feature in enumerate(top_10_features['Feature']):
    plt.subplot(5, 2, i + 1)
    # Filtrar los datos para autistas y controles para la característica actual
    autistas = df[df['condicion'] == 1][feature]
    controles = df[df['condicion'] == 0][feature]

    # Graficar la distribución de los autistas
    sns.kdeplot(autistas, label='Autistas', color='red', fill=True)
    # Graficar la distribución de los controles
    sns.kdeplot(controles, label='Controles', color='blue', fill=True)

    # Configuraciones del título y leyenda del gráfico
    plt.title(f'Distribución de {feature}')
    plt.xlabel('Valor')
    plt.ylabel('Densidad')
    plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(15, 20))

for i, feature in enumerate(top_10_features['Feature']):
    plt.subplot(5, 2, i + 1)

    autistas = df[df['condicion'] == 1][feature]
    controles = df[df['condicion'] == 0][feature]

    sns.kdeplot(autistas, color='red', fill=True)
    sns.kdeplot(controles, color='blue', fill=True)

    # Quita los bordes del gráfico
    sns.despine(left=True, bottom=True)

    # Elimina los ticks de los ejes
    plt.xticks([])
    plt.yticks([])

    # Sin título ni etiquetas de eje
    # plt.title(feature)  # Omitido para no incluir títulos
    # plt.xlabel(feature)  # Omitido para no incluir etiquetas en el eje X

plt.tight_layout()
plt.show()


# Guardar la imagen con fondo transparente
plt.savefig('distr_1.png', transparent=True)

# Descargar la imagen a tu computadora
files.download('distr_1.png')

In [ ]:
### LO ANTERIOR TODO CORRECTO TODO LEGAL

# Logistic Regression with SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

# Verificar la distribución de las clases después del resampling
print(np.bincount(y_train_res))

In [ ]:
from collections import Counter


print("Distribución original de clases: {}".format(Counter(y_train)))

print("Distribución después del resampling: {}".format(Counter(y_train_res)))


Dummy

In [ ]:
from sklearn.dummy import DummyClassifier
dummy_clf = DummyClassifier(strategy="stratified")

dummy_clf.fit(X_train_res, y_train_res)
y_pred = dummy_clf.predict(X_test)

baseline_accuracy = dummy_clf.score(X_test, y_test)
print(f"Baseline accuracy: {baseline_accuracy}")

In [ ]:
conf_mat = confusion_matrix(y_test, y_pred)
print(conf_mat)
sns.heatmap(conf_mat, annot = True, fmt = 'd', cmap= 'Blues')
plt.xlabel('Predicciones del modelo')
plt.ylabel('Condición real')
plt.show()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, make_scorer, roc_auc_score
from sklearn.metrics import classification_report


# Modelo LR con hiperparametros elegidos en base a pruebas anteriores


LR = LogisticRegression(C=0.1, max_iter=1000, solver='newton-cg')


# Define the scoring metrics for cross-validation
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'AUC': 'roc_auc'
}

# Perform cross-validation and save the fitted estimators
cv_results = cross_validate(LR, X_train_res, y_train_res, scoring=scoring, n_jobs=1, verbose=1, return_train_score=True, return_estimator=True)

# Initialize an empty list to store the coefficients
coefs = []

# Iterate through the fitted estimators and extract the coefficients
for estimator in cv_results['estimator']:
    coefs.append(estimator.coef_.ravel())

# Convert the list of coefficients to a NumPy array
coefs_array = np.array(coefs)

# Transpose the array so that each column represents a set of coefficients from one iteration of CV
# coefs_array = coefs_array.T

# Convert the array to a DataFrame
coefs_df = pd.DataFrame(coefs_array)

# Save the DataFrame to an Excel file
coefs_df.to_excel('coeficientes_cv.xlsx', index=False)

# Print the average accuracy and AUC scores from cross-validation
print(f"Average Accuracy: {cv_results['test_accuracy'].mean()}")
print(f"Average AUC: {cv_results['test_AUC'].mean()}")




# Encontrar el índice del mejor modelo
best_index = np.argmax(cv_results['test_accuracy'])

# Seleccionar el mejor modelo
best_model = cv_results['estimator'][best_index]

# Aplicar predict_proba en el conjunto de prueba
probabilities = best_model.predict_proba(X_test)

# Predice las etiquetas del conjunto de prueba
y_pred = best_model.predict(X_test)


# Informe de clasificación
report = classification_report(y_test, y_pred, output_dict=True)

# Resultados
result = {
    'Model': best_model,
    'Accuracy': cv_results['test_accuracy'][best_index],
    'Classification Report': report,
    'Predictions Probabilities': probabilities
}

print("Model:", result['Model'])
print("Accuracy:", result['Accuracy'])
print("Classification Report:", result['Classification Report'])

# print(result)


In [ ]:
# Matriz de confusión con las predicciónes del mejor modelo obtenido
conf_mat = confusion_matrix(y_test, y_pred)
print(conf_mat)
sns.heatmap(conf_mat, annot = True, fmt = 'd', cmap= 'Blues')
plt.xlabel('Predicciones del modelo')
plt.ylabel('Condición real')
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc

y_score = probabilities[:, 1]

# Calcular los valores de tasa de verdaderos positivos (TPR), tasa de falsos positivos (FPR) y los umbrales
fpr, tpr, thresholds = roc_curve(y_test, y_score)

# Calcular el área bajo la curva ROC (AUC)
roc_auc = auc(fpr, tpr)

# Graficar la curva ROC
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
coefs_df

In [ ]:
results_df = add_model_results(results_df, best_model, report, cv_results['test_accuracy'][best_index], cv_results['test_AUC'].mean())

In [ ]:
results_df.head()

In [ ]:
results_df.to_csv('results_2', index=False)

In [ ]:
# Copiar nombres de columnas desde `df` excluyendo 'condicion' si es aplicable
coefs_df.columns = df.columns[:-1]  # Excluye 'condicion' si `df` incluye esta columna
coefs_df.to_csv('coefs_2', index=False)

In [ ]:
# https://forecastegy.com/posts/feature-importance-in-logistic-regression/
# Convertir la lista de coeficientes en una matriz
coefs_matrix = np.array(coefs_df)

# Calcular la mediana para cada columna
median_coefs = np.median(coefs_matrix, axis=0)

# Los coeficientes representan el cambio en el logaritmo de odds por un incremento unitario en la característica correspondiente,
# y por lo tanto, su magnitud (ignorando el signo) puede interpretarse como la "importancia" de la característica en la predicción del modelo.

# Crear DataFrame de importancias
feature_importances = pd.DataFrame({
    'Feature': coefs_df.columns,  # TBD corroborar que realmente se mantenga el orden original
    'Importance': np.abs(median_coefs)
})

# Ordenar las características por importancia
sorted_feature_importances = feature_importances.sort_values(by='Importance', ascending=False)
# Extraer las 10 características más importantes con sus importancias
top_10_features = sorted_feature_importances.head(10)

top_10_features.to_csv('top_10_features_2.csv', index=False)
files.download('top_10_features_2.csv')

# Visualizar las importancias ordenadas
print(sorted_feature_importances)
print(top_10_features)

In [ ]:
# Configurar el tamaño y estilo del gráfico
plt.figure(figsize=(10, 8))
sns.barplot(x='Importance', y='Feature', data=top_10_features, palette='viridis')

# Títulos y etiquetas
plt.title('Top 10 Most Important Features')
plt.xlabel('Importance (Median Absolute Coefficient)')
plt.ylabel('Features')

# Mostrar el gráfico
plt.show()

In [ ]:
plt.figure(figsize=(15, 20))

for i, feature in enumerate(top_10_features['Feature']):
    plt.subplot(5, 2, i + 1)
    # Filtrar los datos para autistas y controles para la característica actual
    autistas = df[df['condicion'] == 1][feature]
    controles = df[df['condicion'] == 0][feature]

    # Graficar la distribución de los autistas
    sns.kdeplot(autistas, label='Autistas', color='red', fill=True)
    # Graficar la distribución de los controles
    sns.kdeplot(controles, label='Controles', color='blue', fill=True)

    # Configuraciones del título y leyenda del gráfico
    plt.title(f'Distribución de {feature}')
    plt.xlabel('Valor')
    plt.ylabel('Densidad')
    plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(15, 20))

for i, feature in enumerate(top_10_features['Feature']):
    plt.subplot(5, 2, i + 1)

    autistas = df[df['condicion'] == 1][feature]
    controles = df[df['condicion'] == 0][feature]

    sns.kdeplot(autistas, color='red', fill=True)
    sns.kdeplot(controles, color='blue', fill=True)

    # Quita los bordes del gráfico
    sns.despine(left=True, bottom=True)

    # Elimina los ticks de los ejes
    plt.xticks([])
    plt.yticks([])

    # Sin título ni etiquetas de eje
    # plt.title(feature)  # Omitido para no incluir títulos
    # plt.xlabel(feature)  # Omitido para no incluir etiquetas en el eje X

plt.tight_layout()
plt.show()


# Guardar la imagen con fondo transparente
plt.savefig('distr_2.png', transparent=True)

# Descargar la imagen a tu computadora
files.download('distr_2.png')

# Logistic Regression with SMOTE and Stratified Shuffle Split

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, make_scorer, roc_auc_score
from sklearn.metrics import classification_report

# Modelo LR con hiperparametros elegidos en base a pruebas anteriores
LR = LogisticRegression(C=0.1, max_iter=1000, solver='newton-cg')

# Define the scoring metrics for cross-validation
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'AUC': 'roc_auc'
}

# Perform cross-validation and save the fitted estimators
cv_results = cross_validate(LR, X_train_res, y_train_res, scoring=scoring, cv=cv_generator, n_jobs=1, verbose=1, return_train_score=True, return_estimator=True)

# Initialize an empty list to store the coefficients
coefs = []

# Iterate through the fitted estimators and extract the coefficients
for estimator in cv_results['estimator']:
    coefs.append(estimator.coef_.ravel())

# Convert the list of coefficients to a NumPy array
coefs_array = np.array(coefs)

# Transpose the array so that each column represents a set of coefficients from one iteration of CV
# coefs_array = coefs_array.T

# Convert the array to a DataFrame
coefs_df = pd.DataFrame(coefs_array)

# Save the DataFrame to an Excel file
coefs_df.to_excel('coeficientes_cv.xlsx', index=False)

# Print the average accuracy and AUC scores from cross-validation
print(f"Average Accuracy: {cv_results['test_accuracy'].mean()}")
print(f"Average AUC: {cv_results['test_AUC'].mean()}")




# Encontrar el índice del mejor modelo
best_index = np.argmax(cv_results['test_accuracy'])

# Seleccionar el mejor modelo
best_model = cv_results['estimator'][best_index]

# Aplicar predict_proba en el conjunto de prueba
probabilities = best_model.predict_proba(X_test)

# Predice las etiquetas del conjunto de prueba
y_pred = best_model.predict(X_test)


# Informe de clasificación
report = classification_report(y_test, y_pred, output_dict=True)

# Resultados
result = {
    'Model': best_model,
    'Accuracy': cv_results['test_accuracy'][best_index],
    'Classification Report': report,
    'Predictions Probabilities': probabilities
}





In [ ]:
# Matriz de confusión con las predicciónes del mejor modelo obtenido

conf_mat = confusion_matrix(y_test, y_pred)
print(conf_mat)
sns.heatmap(conf_mat, annot = True, fmt = 'd', cmap= 'Blues')
plt.xlabel('Predicciones del modelo')
plt.ylabel('Condición real')
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc

y_score = probabilities[:, 1]

# Calcular los valores de tasa de verdaderos positivos (TPR), tasa de falsos positivos (FPR) y los umbrales
fpr, tpr, thresholds = roc_curve(y_test, y_score)

# Calcular el área bajo la curva ROC (AUC)
roc_auc = auc(fpr, tpr)

# Graficar la curva ROC
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
coefs_df

In [ ]:

results_df = add_model_results(results_df, best_model, report, cv_results['test_accuracy'][best_index], cv_results['test_AUC'].mean())

In [ ]:
results_df.head()

In [ ]:
results_df.to_csv('results_3', index=False)

In [ ]:
# Copiar nombres de columnas desde `df` excluyendo 'condicion' si es aplicable
coefs_df.columns = df.columns[:-1]  # Excluye 'condicion' si `df` incluye esta columna
coefs_df.to_csv('coefs_3', index=False)

In [ ]:
# https://forecastegy.com/posts/feature-importance-in-logistic-regression/
# Convertir la lista de coeficientes en una matriz
coefs_matrix = np.array(coefs_df)

# Calcular la mediana para cada columna
median_coefs = np.median(coefs_matrix, axis=0)

# Los coeficientes representan el cambio en el logaritmo de odds por un incremento unitario en la característica correspondiente,
# y por lo tanto, su magnitud (ignorando el signo) puede interpretarse como la "importancia" de la característica en la predicción del modelo.

# Crear DataFrame de importancias
feature_importances = pd.DataFrame({
    'Feature': coefs_df.columns,  # TBD corroborar que realmente se mantenga el orden original
    'Importance': np.abs(median_coefs)
})

# Ordenar las características por importancia
sorted_feature_importances = feature_importances.sort_values(by='Importance', ascending=False)
# Extraer las 10 características más importantes con sus importancias
top_10_features = sorted_feature_importances.head(10)

top_10_features.to_csv('top_10_features_3.csv', index=False)
files.download('top_10_features_3.csv')

# Visualizar las importancias ordenadas
print(sorted_feature_importances)
print(top_10_features)

In [ ]:
# Configurar el tamaño y estilo del gráfico
plt.figure(figsize=(10, 8))
sns.barplot(x='Importance', y='Feature', data=top_10_features, palette='viridis')

# Títulos y etiquetas
plt.title('Top 10 Most Important Features')
plt.xlabel('Importance (Median Absolute Coefficient)')
plt.ylabel('Features')

# Mostrar el gráfico
plt.show()

In [ ]:
plt.figure(figsize=(15, 20))

for i, feature in enumerate(top_10_features['Feature']):
    plt.subplot(5, 2, i + 1)
    # Filtrar los datos para autistas y controles para la característica actual
    autistas = df[df['condicion'] == 1][feature]
    controles = df[df['condicion'] == 0][feature]

    # Graficar la distribución de los autistas
    sns.kdeplot(autistas, label='Autistas', color='red', fill=True)
    # Graficar la distribución de los controles
    sns.kdeplot(controles, label='Controles', color='blue', fill=True)

    # Configuraciones del título y leyenda del gráfico
    plt.title(f'Distribución de {feature}')
    plt.xlabel('Valor')
    plt.ylabel('Densidad')
    plt.legend()

plt.tight_layout()
plt.show()

plt.savefig('distr_3.png', transparent=True)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(15, 20))

for i, feature in enumerate(top_10_features['Feature']):
    plt.subplot(5, 2, i + 1)

    autistas = df[df['condicion'] == 1][feature]
    controles = df[df['condicion'] == 0][feature]

    sns.kdeplot(autistas, color='red', fill=True)
    sns.kdeplot(controles, color='blue', fill=True)

    # Quita los bordes del gráfico
    sns.despine(left=True, bottom=True)

    # Elimina los ticks de los ejes
    plt.xticks([])
    plt.yticks([])

    # Sin título ni etiquetas de eje
    # plt.title(feature)  # Omitido para no incluir títulos
    # plt.xlabel(feature)  # Omitido para no incluir etiquetas en el eje X

plt.tight_layout()
plt.show()

plt.savefig('distr_3_simple.png', transparent=True)
